# Deploying scikit-learn models into production with dotscience

With Dotscience you can build and deploy scikit learn models, and monitor the performance of these models. 

In [ ]:
import dotscience as ds
import os
import shutil
import sklearn
from sklearn import svm
from sklearn import datasets
from joblib import dump
import json

Let's create a very simple model

In [ ]:
clf = svm.SVC(gamma='scale', probability=True)
iris = datasets.load_iris()
X, y = iris.data, iris.target
clf.fit(X, y)

Dump the model as a joblib file, and write a classes.json file mapping class index to its name


In [ ]:
ds.start()

MODEL_DIR = "./model.joblib"

print('export_path = {}\n'.format(MODEL_DIR))

if os.path.isdir(MODEL_DIR):
  print('\nAlready saved a model, cleaning up\n')
  shutil.rmtree(MODEL_DIR)

os.mkdir(MODEL_DIR)

dump(clf, os.path.join(MODEL_DIR, 'model.joblib'))


with open(os.path.join(MODEL_DIR, 'classes.json'), "w") as f:
    f.write(json.dumps({
        "0": "Iris Setosa",
        "1": "Iris Versicolour",
        "2": "Iris Virginica",
    }))

shutil.copyfile("custom_predict.py", os.path.join(MODEL_DIR, "custom_predict.py"))
shutil.copyfile("runtime-requirements.txt", os.path.join(MODEL_DIR, "runtime-requirements.txt"))

Track the model.joblib file with ds.output() and annotate it with ds.model() to tell Dotscience that there is a model. More infromation about the usage of ds.model can be found at https://docs.dotscience.com/references/dotscience-python-library/#models

In [ ]:
ds.model(sklearn, "irisCustom", ds.output(MODEL_DIR), classes="./model.joblib/classes.json")

Publish the model

In [ ]:
ds.publish("trained iris model", deploy=True)